In [13]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from slack_interactor import SlackInteractor
from claude_llm import ClaudeLLM
from db import ActionDatabase
from config import CONFIG
from agent_interface import BaseAgent
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from drunk_agent import DrunkAgent
from runner import process_threads, execute_due_actions

In [15]:
workspaces = CONFIG['workspaces']
slack_interactors = {
    workspace_config['name']: SlackInteractor(workspace_config)
    for _, workspace_config in workspaces.items()
}

agents = {}
for workspace_name, slack_interactor in slack_interactors.items():
    action_db = ActionDatabase(workspace_name)
    agents[workspace_name] = [
        ProjectManagerAgent("claude", action_db, slack_interactor, workspace_name=workspace_name),
        SarcasticAgent("openai", action_db, slack_interactor, workspace_name=workspace_name),
        PaulGrahamAgent("claude", action_db, slack_interactor, workspace_name=workspace_name),
        DrunkAgent("openai", action_db, slack_interactor, workspace_name=workspace_name)
    ]

In [ ]:
for workspace_name, slack_interactor in slack_interactors.items():
    print(f"\nFetching new messages for workspace: {workspace_name}")
    data = slack_interactor.fetch_new_user_messages()
    threads = slack_interactor.organize_threads(data)
    print(f"Found {len(threads)} threads with new user messages in {workspace_name}.")

    if not slack_interactor.is_first_run:
        results = process_threads(agents[workspace_name], threads)
        print(f"\nChecking for due actions in {workspace_name}...")
        execute_due_actions(agents[workspace_name])
    else:
        print(f"First run for {workspace_name}. Skipping thread processing and due actions.")
        slack_interactor.is_first_run = False


Fetching new messages for workspace: agentflow
Set conversations_oldest to 1722399936.894109
Loaded 548 old messages
Found 1 new messages
Updated complete_conversations_agentflow.pkl with new messages
Found 1 threads with new user messages in agentflow.

Processing thread in channel: agentflow
Thread timestamp: 2024-09-03 19:58:13.801378965
Last message content:
geez
Claude claude-3-opus-20240229 prompt:

        You are an AI agent bot named Agentflow with the following characteristics:
        Personality: Professional and efficient
        Goal: Close the loop on open items, nudging and reminding people when necessary. Strive to not be very chatty, and only chime in efficiently.
        Username: PM Agent

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Immediate actions: Tasks that need to be done 

Posted reply to thread 2024-09-03 19:58:13.801378965 in channel agentflow

Executed immediate action for Sarcastic Agent: Executed immediate action: Address Eli's frustration with a light, humorous, and sarcastic remark to ease the tension.
